In [198]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

from datetime import datetime, date, timedelta
from time import sleep
import numpy as np
import pandas as pd
import re
from re import search

In [14]:
# Scraper Tool
def scrape():
    temp = []
    time = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
    contents =  driver.find_elements_by_class_name("tip-item")
    for content in contents:
        temp.append(content.text + f"\n{time}")
    return temp[8:]

# BJ's Data Collection

In [2]:
# PATH = "C:\Program Files (x86)\chromedriver.exe"
# driver = webdriver.Chrome(PATH)
# driver.get("https://www.groupon.com/deals/n-bjs-wholesale-club-memberships") #BJ's
# driver.maximize_window()

# sleep(4)

# all_reviews = driver.find_element_by_xpath("/html/body/section[2]/div/section/div/div[2]/div[2]/section/div/div/article/div[3]/div[2]/div[2]/a")

# all_reviews.click()

In [ ]:
# temp = []
# for i in range(535): #535 Pages
    
#     for j in scrape():
#         temp.append(j)
#     next_button = driver.find_element_by_xpath("/html/body/div[*]/div[2]/div[5]/div[2]")
#     next_button.click()

#     reviews_bj = pd.DataFrame(temp, columns= ["full"])

#     sleep(3)

# reviews = reviews_bj.copy()

In [387]:
# reviews["split"] =reviews["full"].str.split("\n")
# reviews["length"] = [len(i) for i in reviews.split]

# # Not all reviews are the same. Some missing elements
# reviews.length.value_counts()
# reviews.loc[reviews.length == 5,"split"]#[4141]
# reviews.loc[reviews.length == 6,"split"]#[600]
# reviews.loc[reviews.length == 7,"split"]#[0]

In [376]:
# reviews["title"] = ""
# reviews["subtitle"] = ""
# reviews["date_published"] = ""
# reviews["text"] = ""
# reviews["date_scraped"] = ""

In [379]:
# # parsing out split to approriate column based on length

# for i, split in enumerate(reviews.split):

#     if reviews.length[i] == 7:
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = split[2]
#         reviews.loc[reviews.index == i,"date_published"] = split[3]
#         reviews.loc[reviews.index == i,"text"] = split[4]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[6]

#     elif len(split) == 6:
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = split[2]
#         reviews.loc[reviews.index == i,"date_published"] = split[3]
#         reviews.loc[reviews.index == i,"text"] = "None"
#         reviews.loc[reviews.index == i,"date_scraped"] =split[5]

#     elif len(split) == 5:
#         reviews.loc[reviews.index == i,"title"] = "None"
#         reviews.loc[reviews.index == i,"subtitle"] = split[0]
#         reviews.loc[reviews.index == i,"date_published"] = split[1]
#         reviews.loc[reviews.index == i,"text"] = split[2]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[4]

In [388]:
# # Found an anomly with reviews with review 4661. Had no name but was a helpful reviewer

# reviews.split[4661]
# reviews.loc[reviews.index == 4661,"title"] = "None"+reviews.split[4661][0]
# reviews.loc[reviews.index == 4661,"subtitle"] = reviews.split[4661][1]
# reviews.loc[reviews.index == 4661,"date_published"] = reviews.split[4661][2]
# reviews.loc[reviews.index == 4661,"text"] = reviews.split[4661][3]
# reviews.loc[reviews.index == 4661,"date_scraped"] = reviews.split[4661][5]
# reviews.loc[reviews.index == 4661,]

In [381]:
# # # Parsing out title and subtitle columns further

# reviews["top_reviewer"] = [1 if ("TOP REVIEWER" in i) else 0 for i in reviews.title]
# reviews["helpful_reviewer"] = [1 if ("HELPFUL REVIEWER" in i) else 0 for i in reviews.title]
# reviews["rating"] = [int(re.findall(r'\d+',i)[0]) if len(re.findall(r'\d+',i))>0 else 0 for i in reviews.subtitle]
# reviews["name"] = [re.split('TOP REVIEWER|HELPFUL REVIEWER',i)[0] for i in reviews.title]

# reviews["review_count"] = 0
# for i, subtitle in enumerate(reviews.subtitle[:10]):
#     num = re.findall(r'\d+',subtitle)
#     if len(num) > 0:
#         if search ("rating", subtitle):
#             reviews.loc[reviews.index == i,"review_count"] = int(num[1])
#         else:
#             reviews.loc[reviews.index == i,"review_count"] = int(num[0])

In [382]:
# # Formating date_published that are in a relative format (# days/hours ago)

# for i, date in enumerate(reviews.date_published):
    
#     if "ago" in date:
#         if "day" in date:
#             scraped = datetime.strptime(reviews.date_scraped[i],"%m/%d/%Y %H:%M:%S").date()
#             ago = timedelta(days = int(re.findall(r'\d+',date)[0]))
#             reviews.loc[reviews.index == i,"date_published"] = (scraped - ago).strftime("%m/%d/%Y")
#         else:
#             reviews.loc[reviews.index == i,"date_published"] = reviews.date_scraped[i][:10]
#     elif date != "'": 
#         reviews.loc[reviews.index == i,"date_published"] = datetime.strptime(reviews.date_published[i],"%B %d, %Y").date().strftime("%m/%d/%Y")


In [385]:
# # BJ's CSV
# reviews.to_csv('Groupon_BJs_Raw.csv', index=False)
# reviews[['name', 'text', 'rating', 'review_count','top_reviewer', 'helpful_reviewer','date_published','date_scraped']].to_csv('Groupon_BJs.csv', index=False)

# Sam's Club Data Collection

In [ ]:
# PATH = "C:\Program Files (x86)\chromedriver.exe"
# driver = webdriver.Chrome(PATH)
# # driver.get("https://www.groupon.com/deals/n-sams-club-membership-packages-7#tips") #Sam's Club
# driver.maximize_window()

# sleep(4)

# all_reviews = driver.find_element_by_xpath("/html/body/section[2]/div/section/div/div[2]/div[2]/section/div/div/article/div[3]/div[2]/div[2]/a")
# all_reviews.click()

In [36]:
# temp = []
# for i in range(2828): #2828
    
#     for j in scrape():
#         temp.append(j)
#     next_button = driver.find_element_by_xpath("/html/body/div[*]/div[2]/div[5]/div[2]")
#     next_button.click()

#     reviews_sam = pd.DataFrame(temp, columns= ["full"])

#     sleep(3)

# reviews = reviews_sam.copy()

In [278]:
# reviews["split"] =reviews["full"].str.split("\n")
# reviews["length"] = [len(i) for i in reviews.split]

In [389]:
# # Not all reviews are the same. Some missing elements
# reviews.length.value_counts()
# reviews.loc[reviews.length == 5,"split"]#[4141]
# reviews.loc[reviews.length == 6,"split"]#[600]
# reviews.loc[reviews.length == 7,"split"]#[0]

In [280]:
# reviews["title"] = ""
# reviews["subtitle"] = ""
# reviews["date_published"] = ""
# reviews["text"] = ""
# reviews["date_scraped"] = ""

In [281]:
# for i, split in enumerate(reviews.split):

#     if len(split) == 7:
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = split[2]
#         reviews.loc[reviews.index == i,"date_published"] = split[3]
#         reviews.loc[reviews.index == i,"text"] = split[4]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[6]

#     elif len(split) == 6:
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = split[2]
#         reviews.loc[reviews.index == i,"date_published"] = split[3]
#         reviews.loc[reviews.index == i,"text"] = "None"
#         reviews.loc[reviews.index == i,"date_scraped"] =split[5]

#     elif len(split) == 5:
#         reviews.loc[reviews.index == i,"title"] = "None"
#         reviews.loc[reviews.index == i,"subtitle"] = split[0]
#         reviews.loc[reviews.index == i,"date_published"] = split[1]
#         reviews.loc[reviews.index == i,"text"] = split[2]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[4]

In [386]:
# Checked reviews 
# reviews.loc[reviews.length == 5,]
# reviews.loc[reviews.length == 6,]
# reviews.loc[reviews.length == 7,]

In [288]:
# # Found an anomly with reviews with length 6. Some have no rating but have text. Some have no text but ratings. Some have no text or ratings but are a TOP/HELPFUL Reviewer.

# no_subtitle = reviews.index[reviews.subtitle.str.contains("rating|reviews") == False].tolist()
# no_title = reviews.index[reviews.title.str.contains("rating")].tolist()

# for i, split in enumerate(reviews.split):

#     if len(split) == 6 and (i in no_subtitle):
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = "None"
#         reviews.loc[reviews.index == i,"date_published"] = split[2]
#         reviews.loc[reviews.index == i,"text"] = split[3]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[5]

#     if len(split) == 6 and (i in no_title):
#         reviews.loc[reviews.index == i,"title"] = "None" + split[0]
#         reviews.loc[reviews.index == i,"subtitle"] = split[1]
#         reviews.loc[reviews.index == i,"date_published"] = split[2]
#         reviews.loc[reviews.index == i,"text"] = split[3]
#         reviews.loc[reviews.index == i,"date_scraped"] = split[5]

# # Check 
# # reviews.loc[reviews.length == 6,]

In [290]:
# # # Parsing out title and subtitle columns further

# reviews["top_reviewer"] = [1 if ("TOP REVIEWER" in i) else 0 for i in reviews.title]
# reviews["helpful_reviewer"] = [1 if ("HELPFUL REVIEWER" in i) else 0 for i in reviews.title]
# reviews["rating"] = [int(re.findall(r'\d+',i)[0]) if len(re.findall(r'\d+',i))>0 else 0 for i in reviews.subtitle]
# reviews["name"] = [re.split('TOP REVIEWER|HELPFUL REVIEWER',i)[0] for i in reviews.title]

# reviews["review_count"] = 0
# for i, subtitle in enumerate(reviews.subtitle[:10]):
#     num = re.findall(r'\d+',subtitle)
#     if len(num) > 0:
#         if search ("rating", subtitle):
#             reviews.loc[reviews.index == i,"review_count"] = int(num[1])
#         else:
#             reviews.loc[reviews.index == i,"review_count"] = int(num[0])

In [305]:
# # Formating date_published that are in a relative format (# days/hours ago)

# for i, date in enumerate(reviews.date_published):
    
#     if "ago" in date:
#         if "day" in date:
#             scraped = datetime.strptime(reviews.date_scraped[i],"%m/%d/%Y %H:%M:%S").date()
#             ago = timedelta(days = int(re.findall(r'\d+',date)[0]))
#             reviews.loc[reviews.index == i,"date_published"] = (scraped - ago).strftime("%m/%d/%Y")
#         else:
#             reviews.loc[reviews.index == i,"date_published"] = reviews.date_scraped[i][:10]
#     else: 
#         reviews.loc[reviews.index == i,"date_published"] = datetime.strptime(reviews.date_published[i],"%B %d, %Y").date().strftime("%m/%d/%Y")


In [309]:
# # # Sam's Club CSV
# reviews.to_csv('Groupon_SamsClub_Raw.csv', index=False)
# reviews[['name', 'text', 'rating', 'review_count','top_reviewer', 'helpful_reviewer','date_published','date_scraped']].to_csv('Groupon_SamsClub.csv', index=False)